In [3]:
import os

import numpy as np
import pandas as pd
import xarray as xr

from rainproj.rainreader import list_available_files
from rainproj.rainreader2 import RainReader2

pd.options.plotting.backend ='plotly'

WORK = os.environ.get('WORK')
SCRATCH = os.environ.get('SCRATCH')

ROOT_DIR = '/scratch/09295/naveens/hindcast'
rr = RainReader2(root_dir=ROOT_DIR,chunks='auto')

In [14]:
accessible_files = list_available_files(ROOT_DIR,run_parallel=True)

Parallel enabled
16802 of 16802 100.0 files accessible


In [15]:
# RainReader2.run_xarray_transforms('/scratch/09295/naveens/hindcast/1979/surface_variables_1979-01-01.nc').date.values#.to_netcdf(f'{SCRATCH}/hindcast/test-surface_variables_1979-01-01.nc')

In [17]:
# xr.open_dataarray(f'{SCRATCH}/hindcast/xarray/stage1/surface_variables_rain_1979-01-01.nc')#.date.values

## Transform Files

In [18]:
from joblib import Parallel, delayed
import os
def execute(filepath):
    out_path = f'{SCRATCH}/hindcast/xarray/stage1/'+filepath.split('/')[-1].replace('surface_variables','surface_variables_rain')
    RainReader2.run_xarray_transforms(filepath).to_netcdf(out_path)
    
parallel = Parallel(n_jobs=os.cpu_count()-1)
silent = parallel(delayed(execute)(file) for file in accessible_files)

## Merge Files

In [4]:
xr.open_dataset('/home1/07989/clin265/SCRATCH/hindcast/xarray/era_stage1/era_1979-07-11.nc')

<xarray.Dataset> Size: 94kB
Dimensions:              (time: 1, lat: 99, lon: 233)
Coordinates:
  * lat                  (lat) float32 396B 25.0 25.25 25.5 ... 49.0 49.25 49.5
  * lon                  (lon) float32 932B 235.0 235.2 235.5 ... 292.8 293.0
  * time                 (time) datetime64[ns] 8B 1979-07-11
Data variables:
    total_precipitation  (time, lat, lon) float32 92kB ...

In [19]:
from pathlib import Path

stage2_dir = Path(f'{SCRATCH}/hindcast/xarray/stage2/')
stage2_dir.mkdir(parents=True,exist_ok=True)

In [ ]:
stage1_files = os.listdir(f'{SCRATCH}/hindcast/xarray/stage1/')

In [21]:
import glob 

year_months = set([filename.split('_')[-1].rsplit('-',maxsplit=1)[0] for filename in stage1_files])
mon_year_files = {prefix:glob.glob(f'{SCRATCH}/hindcast/xarray/stage1/*{prefix}*.nc') for prefix in year_months}

In [22]:
from joblib import Parallel, delayed
import os
def execute1(prefix_file_batch):
    prefix,file_batch = prefix_file_batch
    out_path = f'{SCRATCH}/hindcast/xarray/stage2/surface_variables_rain_{prefix}.nc'
    xr.open_mfdataset(file_batch,combine='nested',concat_dim='date',engine="netcdf4").sortby('date').to_netcdf(out_path)
parallel = Parallel(n_jobs=os.cpu_count()-1)
silent2 = parallel(delayed(execute1)(prefix_file_batch) for prefix_file_batch in mon_year_files.items())

/tmp/ipykernel_3314179/3295761025.py:6: FutureWarning: In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): 'lead_time' ('lead_time',) The recommendation is to set join explicitly for this case.
